In [1]:
import langchain
from langchain.llms import VertexAI
import streamlit as st
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [2]:
import pandas as pd 
import re 
from bs4 import BeautifulSoup
from langchain.docstore.document import Document 
from langchain.embeddings import VertexAIEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import os 
from langchain.vectorstores import DocArrayInMemorySearch

In [3]:
from google.cloud import aiplatform

In [4]:
aiplatform.init(project="gcp-cxbcsdev-nprd-34905")

In [5]:
llm = VertexAI(temperature=0.8)

In [6]:
from bs4 import BeautifulSoup

In [7]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()   
    clean_doc = re.sub(r'\xa0', '', text)
    return clean_doc

In [8]:
import pandas as pd
import re 
from langchain.docstore.document import Document 

In [9]:
class_name = 'Licensing'

In [10]:
df = pd.read_excel('dataset/tag_clusters.xlsx')

In [11]:
# filter for annual analysis window, class name, and remove nan values
df = df[df['analysis_window'] == 'annual']
df = df[(df['class'] == class_name)]
df['text'] = df['text'].apply(remove_html_tags)

In [12]:
docs = df['text'].tolist()
post_url = df['post_url'].tolist()
post_tag = df['tag'].tolist()
post_topic = df['topic_name'].tolist()
post_summary = df['summary_text'].tolist()
conversation_id = df['conversation_id'].tolist()

In [13]:
# convert nan values to ''
docs = [doc if isinstance(doc, str) else '' for doc in docs]
post_summary = [summary if isinstance(summary, str) else '' for summary in post_summary]

In [14]:
try:

    doc_augmented = [post_summary + " " + doc for post_summary, doc in zip(post_summary, docs)]

except TypeError:
    print("TypeError encountered")
    print("post_summary: ", post_summary)
    print("doc: ", docs)

docs = [Document(page_content=doc) for doc in doc_augmented]

                    # metadata={'url': post_url[i], 
                    #         'tag': post_tag[i], 
                    #         'topic_name': post_topic[i], 
                    #         'summary': post_summary[i], 
                    #         'conversation_id': conversation_id[i]}) for i, doc in enumerate(doc_augmented)]

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20,
        separators=['||||']
)

docs = text_splitter.split_documents(docs)
print("Documents split <<>>")

embeddings = VertexAIEmbeddings()

Documents split <<>>


In [16]:
db_name = class_name

In [17]:
from langchain.vectorstores import DocArrayInMemorySearch

In [18]:
docs[0]

Document(page_content='The issue being faced is that the person is not sure if they need to purchase ISE Advantage licenses in addition to the DNA Advantage licenses they have already purchased. They are concerned because the Cisco ISE 3.x licensing structure states that ISE Advantage licenses are required to use pxGrid and TrustSec. Hello Folks,\n\nQuick question. I have purchased DNA Advantage licenses for Cisco SD-Access deployment. With SD-Access, we already have ISE 3.x Essential licenses. The question is do I also', metadata={})

In [19]:
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

TypeError: Cannot create a consistent method resolution
order (MRO) for bases type, ABCMeta

In [20]:
docs

[Document(page_content='The issue being faced is that the person is not sure if they need to purchase ISE Advantage licenses in addition to the DNA Advantage licenses they have already purchased. They are concerned because the Cisco ISE 3.x licensing structure states that ISE Advantage licenses are required to use pxGrid and TrustSec. Hello Folks,\n\nQuick question. I have purchased DNA Advantage licenses for Cisco SD-Access deployment. With SD-Access, we already have ISE 3.x Essential licenses. The question is do I also', metadata={}),
 Document(page_content='uestion is do I also need to purchase ISE Advantage license? As mentioned on Cisco ISE 3.x licensing structure, it says that to use pxGrid (with DNA-C - no pxGrid Cloud required in my case) and TrustSec (Group-based Access Policy), I need ISE advantage licenses. Can someone please clarify based on their experience. |||| check this matrix will helo you depends what feature you looking to implement :\nhttps://www.cisco.com/c/m/en_u

In [6]:
df.columns

Index(['topic', 'doc', 'post_x_coordinate', 'post_y_coordinate', 'user',
       'topic_id', 'topic_name', 'post_url', 'post_preview', 'class',
       'analysis_window', 'quarter_for_analysis', 'conversation_id', 'text',
       'summary_text', 'tag'],
      dtype='object')

In [41]:
from bs4 import BeautifulSoup
import requests

In [13]:
df['class'].value_counts()

class
Implement             667
Design/Integration    191
Upgrade               107
Licensing              74
Name: count, dtype: int64

In [14]:
df.columns

Index(['topic', 'doc', 'post_x_coordinate', 'post_y_coordinate', 'user',
       'topic_id', 'topic_name', 'post_url', 'post_preview', 'class',
       'analysis_window', 'quarter_for_analysis', 'conversation_id', 'text',
       'summary_text', 'tag'],
      dtype='object')

In [15]:
df = df[(df['class'] == class_name) & (df['analysis_window'] == 'annual')]

In [16]:
df.shape

(74, 16)

In [206]:
from bs4 import BeautifulSoup
def remove_html_tags(text):
        soup = BeautifulSoup(text, 'html.parser')
        return soup.get_text()

In [207]:
docs = [remove_html_tags(d) if isinstance(d, str) else '' for d in df['text'].values]

In [208]:
for d in docs:
    if not isinstance(d, str):
        print(d)

In [209]:
import re 

docs = [re.sub(r'\xa0', '', doc) for doc in docs]

In [210]:
df['text'] = docs

In [211]:
docs = ' '.join([d if isinstance(d, str) else '' for d in docs])
docs

'Hello Folks,\n\nQuick question. I have purchased DNA Advantage licenses for Cisco SD-Access deployment. With SD-Access, we already have ISE 3.x Essential licenses. The question is do I also need to purchase ISE Advantage license? As mentioned on Cisco ISE 3.x licensing structure, it says that to use pxGrid (with DNA-C - no pxGrid Cloud required in my case) and TrustSec (Group-based Access Policy), I need ISE advantage licenses. Can someone please clarify based on their experience. |||| check this matrix will helo you depends what feature you looking to implement :\nhttps://www.cisco.com/c/m/en_uk/products/software/dna-subscription-switching/en-sw-sub-matrix-switching.html Hello All,\nStarting to get into ISE and DNA and a couple of queries if someone can point me in the right direction. There is a lot of content on both subjects and I cant quite piece together where DNA Center is necessary when it comes to SDA. For example do I need DNA Center to implement Trustsec, SGT\'s and Posture

In [212]:
 # Split documents into chunks


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=10,  separators=["\n\n", '\n']
)
texts = text_splitter.split_text(docs)

In [213]:
texts[:20]

['Hello Folks,',
 'Quick question. I have purchased DNA Advantage licenses for Cisco SD-Access deployment. With SD-Access, we already have ISE 3.x Essential licenses. The question is do I also need to purchase ISE Advantage license? As mentioned on Cisco ISE 3.x licensing structure, it says that to use pxGrid (with DNA-C - no pxGrid Cloud required in my case) and TrustSec (Group-based Access Policy), I need ISE advantage licenses. Can someone please clarify based on their experience. |||| check this matrix will helo you depends what feature you looking to implement :\nhttps://www.cisco.com/c/m/en_uk/products/software/dna-subscription-switching/en-sw-sub-matrix-switching.html Hello All,',
 "Starting to get into ISE and DNA and a couple of queries if someone can point me in the right direction. There is a lot of content on both subjects and I cant quite piece together where DNA Center is necessary when it comes to SDA. For example do I need DNA Center to implement Trustsec, SGT's and Pos

In [222]:
df.head()

,topic,doc,post_x_coordinate,post_y_coordinate,user,topic_id,topic_name,post_url,post_preview,class,analysis_window,quarter_for_analysis,conversation_id,text,summary_text,tag
0,0,person purchase advantage license additio...,0.380220,0.056250,2164,0,0_advantage_license_dna,https://community.cisco.com/t5/cisco-one-discu...,"Hello Folks,\n&nbsp;\nQuick question. I have p...",Licensing,annual,20233,4757953,"Hello Folks,\n\nQuick question. I have purchas...",The issue being faced is that the person is no...,Cisco dna
1,0,understand relationship dna center sda whet...,0.738165,0.836678,23114,0,0_advantage_license_dna,https://community.cisco.com/t5/cisco-digital-n...,"Hello All,\nStarting to get into ISE and DNA a...",Licensing,annual,20233,4725319,"Hello All,\nStarting to get into ISE and DNA a...",The user is trying to understand the relations...,Cisco dna
2,0,enable policy based enforcement essentia...,0.234175,0.935613,73380,0,0_advantage_license_dna,https://community.cisco.com/t5/software-define...,I have question about ISE&nbsp;I have ISE esse...,Licensing,annual,20233,4614950,I have question about ISEI have ISE essential ...,The issue being faced is that the user wants t...,Cisco dna
3,0,generating alarm smart authorization renew...,0.590524,0.374223,63672,0,0_smart_failure_authorization,https://community.cisco.com/t5/network-access-...,"Hi guys,&nbsp;I am running two node deployment...",Licensing,annual,20233,4638108,"Hi guys,I am running two node deployment with ...",The issue being faced is that ISE is generatin...,Cisco firewall
4,0,smart license box connected internet able o...,0.152381,0.551912,84584,0,0_smart_failure_authorization,https://community.cisco.com/t5/network-access-...,I want to smart license my ISE box which ia no...,Licensing,annual,20233,4613090,I want to smart license my ISE box which ia no...,The customer wants to smart license their ISE ...,Cisco firewall


In [ ]:
docs = df['text'].tolist()
post_url = df['post_url'].tolist()
post_tag = df['tag'].tolist()
post_topic = df['topic_name'].tolist()

In [214]:
from langchain.docstore.document import Document 

docs = [Document(page_content=doc, metadata={'url': post_url[i], 'tag': post_tag[i], 'topic_name': post_topic[i]}) for i, doc in enumerate(texts)]

In [5]:
# embeddings = VertexAIEmbeddings()
# Create a vectorstore from documents
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch


embeddings = VertexAIEmbeddings()

# db = DocArrayInMemorySearch.from_documents(
#     docs, 
#     embeddings
# )

db = Chroma.from_documents(docs, embeddings, persist_directory=f'vector_databases/{class_name}_db')
db.persist()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\farefain\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-d09f1092ac28>", line 14, in <module>
    db = Chroma.from_documents(docs, embeddings, persist_directory=f'vector_databases/{class_name}_db')
NameError: name 'docs' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\farefain\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\farefain\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\ultr

TypeError: object of type 'NoneType' has no len()

In [217]:

db._client_settings.persist_directory

'vector_databases/Licensing_db'

In [44]:
query = "Give me all the issues about DACL discussed among customers. Provide them in bullet points."

In [311]:
retriever = db.as_retriever

In [326]:
retriever.get_relevant_documents(query=query)

[Document(page_content="I have noticed one weird bug in ISE when the CoA is not working (i.e. when it's not configured on the switch) - the result is that the session is correctly authorized, but you will see no entry in Li", metadata={}),
 Document(page_content='If you have already this , and you are seeing the events related to the passive ID connector, I would check if the ISE and the NAD are issuing the CoA correctly.', metadata={}),
 Document(page_content='ng for a long time. I suspect this may have something to do with the problem perhaps. Having looked at the destination of the CoA packets, it is clear that ISE is trying to ask the original APs based', metadata={}),
 Document(page_content='ation packets to construct CoA? If so, how does that explain the alarm messages I have been getting on ISE complaining about lack of response to generated CoA to APs? Hello,', metadata={})]

In [312]:
query = 'Give me more context into issues customers are having where switche is not receiving the correct CoA from the ISE.'

In [318]:
c = db.similarity_search(query=query, k=8)

In [319]:
context = '\n\n'.join([page.page_content for page in c])

In [331]:
db._persist_directory

'dataset/Implement_db'

In [14]:
class_name = 'upgrade'

In [55]:
query = 'list all ISE implement issues involding Cisco dna discussed in bullet points.'

In [6]:
query = "What type of questions are getting asked on licenses for Cisco SD-Access deployment"

In [9]:
persist_directory = './community-app/vector_databases/Licensing_db'

embeddings = VertexAIEmbeddings()

vectordb = Chroma(persist_directory=persist_directory, 
                      embedding_function=embeddings)


In [16]:
# Check validity of the imported database
vectordb._client_settings.__dict__()

TypeError: 'dict' object is not callable

In [8]:
vectordb.similarity_search(query=query, k=4)

[Document(page_content='The issue being faced is that the person is not sure if they need to purchase ISE Advantage licenses in addition to the DNA Advantage licenses they have already purchased. They are concerned because the Cisco ISE 3.x licensing structure states that ISE Advantage licenses are required to use pxGrid and TrustSec. Hello Folks,\n\nQuick question. I have purchased DNA Advantage licenses for Cisco SD-Access deployment. With SD-Access, we already have ISE 3.x Essential licenses. The question is do I also', metadata={'url': 'https://community.cisco.com/t5/cisco-one-discussions/cisco-dna-advantage-for-sd-access-and-ise-advantage-license/td-p/4757953', 'tag': 'Cisco dna', 'topic_name': '0_advantage_license_dna', 'summary': 'The issue being faced is that the person is not sure if they need to purchase ISE Advantage licenses in addition to the DNA Advantage licenses they have already purchased. They are concerned because the Cisco ISE 3.x licensing structure states that ISE

In [8]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.llms import OpenAI 

In [9]:
metadata_field_info = [
    AttributeInfo(
        name = 'url',
        description='Where the document is from. Should be a link to the document',
        type = 'string'
    ),
    AttributeInfo(
        name = 'tag',
        description= "What subc-class the document belongs too. Should be one of 'Others', 'Directory services', 'Cloud/OS', 'Integration', 'Cisco firewall', 'Authentication', 'Cisco dna', 'Design'",
        type = 'string'
    ),
    AttributeInfo(
        name = 'topic_name',
        description = "The topic where the document belongs to.",
        type = 'sstring'

    )
]

In [14]:
document_content_description = "ISE product category documents"

llm = VertexAI(temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm=llm, 
    vectorstore=vectordb,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

In [15]:
query = "Give me discussions where a  customer wants to smart license their ISE box which is not connected to the internet."

In [16]:
results = retriever.get_relevant_documents(query=query)

c:\Users\farefain\AppData\Local\Programs\Python\Python38\lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='smart license ISE box not connected to the internet' filter=None limit=None


In [23]:
results[0].metadata['url']

'https://community.cisco.com/t5/network-access-control/ise-smart-licensing/td-p/4613090'

In [35]:
d = pd.read_excel('dataset/cluster_summaries_aq.xlsx')

In [55]:
len(d[(d['class'] == 'Licensing') & (d['tag'] == 'Others') & (d['analysis_window'] == 'annual')])

2

In [32]:
df.head()

,topic,doc,post_x_coordinate,post_y_coordinate,user,topic_id,topic_name,post_url,post_preview,class,analysis_window,quarter_for_analysis,conversation_id,text,summary_text,tag
0,0,person purchase advantage license additio...,0.380220,0.056250,2164,0,0_advantage_license_dna,https://community.cisco.com/t5/cisco-one-discu...,"Hello Folks,\n&nbsp;\nQuick question. I have p...",Licensing,annual,20233,4757953,"<P>Hello Folks,</P>\n<P>&nbsp;</P>\n<P>Quick q...",The issue being faced is that the person is no...,Cisco dna
1,0,understand relationship dna center sda whet...,0.738165,0.836678,23114,0,0_advantage_license_dna,https://community.cisco.com/t5/cisco-digital-n...,"Hello All,\nStarting to get into ISE and DNA a...",Licensing,annual,20233,4725319,"<P>Hello All,</P>\n<P>Starting to get into ISE...",The user is trying to understand the relations...,Cisco dna
2,0,enable policy based enforcement essentia...,0.234175,0.935613,73380,0,0_advantage_license_dna,https://community.cisco.com/t5/software-define...,I have question about ISE&nbsp;I have ISE esse...,Licensing,annual,20233,4614950,<P>I have question about ISE</P><P>&nbsp;</P><...,The issue being faced is that the user wants t...,Cisco dna
3,0,generating alarm smart authorization renew...,0.590524,0.374223,63672,0,0_smart_failure_authorization,https://community.cisco.com/t5/network-access-...,"Hi guys,&nbsp;I am running two node deployment...",Licensing,annual,20233,4638108,"<P>Hi guys,</P><P>&nbsp;</P><P>I am running tw...",The issue being faced is that ISE is generatin...,Cisco firewall
4,0,smart license box connected internet able o...,0.152381,0.551912,84584,0,0_smart_failure_authorization,https://community.cisco.com/t5/network-access-...,I want to smart license my ISE box which ia no...,Licensing,annual,20233,4613090,<P>I want to smart license my ISE box which ia...,The customer wants to smart license their ISE ...,Cisco firewall


In [28]:
list('Design/')

['D', 'e', 's', 'i', 'g', 'n', '/']

In [31]:
'Design'.split('/')

['Design']

In [27]:
vectordb.similarity_search(query, k=3)

[Document(page_content='worth looking suggested steps on this thread :\nhttps://community.cisco.com/t5/wireless/cisco-ise-guest-portal-redirect/td-p/3786304 |||| Sorry, we need a lot more detail here.\nWhat would you need to tell the TAC to help you troubleshoot this?\nIt would be nice if you shared the ISE LiveLog details or errors.\nExplain why it is not what you expected.\xa0\nWhat did you expect?\xa0\xa0\nWhat ISE authorization profile was assigned? You may need to share the details of the assignment depending on the issue.', metadata={'url': 'https://community.cisco.com/t5/network-access-control/cisco-ise-for-external-guest-access/td-p/4669839', 'tag': 'Others', 'topic_name': '1_device_network_portal'}),
 Document(page_content='Good morning,I have a user in AD who is blocked all time in Cisco ISE (Screenshot 1 and 2).Firstly, i had this issue "24415 User authentication against AD failed since user\'s account is locked out"(Screenshot 3).I changed some configurations (Screenshot 4)

In [320]:
from pprint import pprint 

pprint(context)

('I have noticed one weird bug in ISE when the CoA is not working (i.e. when '
 "it's not configured on the switch) - the result is that the session is "
 'correctly authorized, but you will see no entry in Li\n'
 '\n'
 'If you have already this , and you are seeing the events related to the '
 'passive ID connector, I would check if the ISE and the NAD are issuing the '
 'CoA correctly.\n'
 '\n'
 'ng for a long time. I suspect this may have something to do with the problem '
 'perhaps. Having looked at the destination of the CoA packets, it is clear '
 'that ISE is trying to ask the original APs based\n'
 '\n'
 'ation packets to construct CoA? If so, how does that explain the alarm '
 'messages I have been getting on ISE complaining about lack of response to '
 'generated CoA to APs? Hello,\n'
 '\n'
 'https://community.cisco.com/t5/wireless/cisco-ise-guest-portal-redirect/td-p/3786304 '
 '|||| Sorry, we need a lot more detail here.\n'
 'What would you need to tell the TAC to help you 

In [89]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [48]:
# results = db.similarity_search(query, k=5)
# for result in results:
#     content.append(result.page_content)
# content

[':\nhttps://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/guide-c07-656177.html\n\xa0 I have a customer who owns ISE admin but not FTD admin so they want to do dACL for RA VPN so they can skip the Access Control Policy on FTD. (So the less they have to ask to the FTD team',
 "base license?Thank you very much for your help and discussion!\xa0thanksJeffery\xa0\xa0\xa0\xa0 |||| Do you have RADIUS Accounting enabled on the ASA?\xa0 I'm not sure of way to forcibly delete these (other than deleting the endpoint from context visibility maybe?).\xa0 What patch of 2.6 are you",
 'below\nhttps://bst.cisco.com/bugsearch/bug/CSCvw34491 Hi all,I have question please.Can I do accounting for device administration and without TACACS device admin license? |||| I mean Commands accounting. not endpoints accounting, my scope is to use ISE only for device administration Hello,I need']

In [124]:
def get_context(db, query):

    related_docs = db.similarity_search(query, k=8)

    context = ' '.join([page.page_content for page in related_docs])

    return context

In [76]:
# c = get_context(db, query)
# c

':\nhttps://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/guide-c07-656177.html\n\xa0 I have a customer who owns ISE admin but not FTD admin so they want to do dACL for RA VPN so they can skip the Access Control Policy on FTD. (So the less they have to ask to the FTD team base license?Thank you very much for your help and discussion!\xa0thanksJeffery\xa0\xa0\xa0\xa0 |||| Do you have RADIUS Accounting enabled on the ASA?\xa0 I\'m not sure of way to forcibly delete these (other than deleting the endpoint from context visibility maybe?).\xa0 What patch of 2.6 are you below\nhttps://bst.cisco.com/bugsearch/bug/CSCvw34491 Hi all,I have question please.Can I do accounting for device administration and without TACACS device admin license? |||| I mean Commands accounting. not endpoints accounting, my scope is to use ISE only for device administration Hello,I need actually working on a project integrating Dell (N Series) NAD\'s into ISE and would appreciate a clean

In [106]:
template = """From the history and context provided below, Answer the question as truthfully as possible. 
    and if the answer is not contained within the context or history below, say 'I don't know' or ask me to refine my query. 

Current conversation:
{history}

{input}
"""


In [107]:
system_msg_template = SystemMessagePromptTemplate.from_template(template=template)


In [108]:
human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")

In [109]:
prompt_template = ChatPromptTemplate.from_messages([system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])

In [110]:
from langchain.chains import ConversationChain
conversation = ConversationChain(llm=llm, prompt=prompt_template, verbose=True, memory=ConversationBufferWindowMemory(k=3,return_messages=True))

In [111]:
conversation.prompt

ChatPromptTemplate(input_variables=['input', 'history'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['history', 'input'], output_parser=None, partial_variables={}, template="From the history and context provided below, Answer the question as truthfully as possible. \n    and if the answer is not contained within the context or history below, say 'I don't know' or ask me to refine my query. \n\nCurrent conversation:\n{history}\n\n{input}\n", template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [112]:
query

'Give me all the issues about DACL discussed among customers. Provide them in bullet points.'

In [113]:
context = get_context(db, query)

In [114]:
context

':\nhttps://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/guide-c07-656177.html\n\xa0 I have a customer who owns ISE admin but not FTD admin so they want to do dACL for RA VPN so they can skip the Access Control Policy on FTD. (So the less they have to ask to the FTD team base license?Thank you very much for your help and discussion!\xa0thanksJeffery\xa0\xa0\xa0\xa0 |||| Do you have RADIUS Accounting enabled on the ASA?\xa0 I\'m not sure of way to forcibly delete these (other than deleting the endpoint from context visibility maybe?).\xa0 What patch of 2.6 are you below\nhttps://bst.cisco.com/bugsearch/bug/CSCvw34491 Hi all,I have question please.Can I do accounting for device administration and without TACACS device admin license? |||| I mean Commands accounting. not endpoints accounting, my scope is to use ISE only for device administration Hello,I need actually working on a project integrating Dell (N Series) NAD\'s into ISE and would appreciate a clean

In [115]:
conversation.predict(input=f"Context:\n ```{context}``` \n\n Query:\n```{query}```")



> Entering new  chain...
Prompt after formatting:
System: From the history and context provided below, Answer the question as truthfully as possible. 
    and if the answer is not contained within the context or history below, say 'I don't know' or ask me to refine my query. 

Current conversation:
[]

Context:
 ```:
https://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/guide-c07-656177.html
  I have a customer who owns ISE admin but not FTD admin so they want to do dACL for RA VPN so they can skip the Access Control Policy on FTD. (So the less they have to ask to the FTD team base license?Thank you very much for your help and discussion! thanksJeffery     |||| Do you have RADIUS Accounting enabled on the ASA?  I'm not sure of way to forcibly delete these (other than deleting the endpoint from context visibility maybe?).  What patch of 2.6 are you below
https://bst.cisco.com/bugsearch/bug/CSCvw34491 Hi all,I have question please.Can I do accounting for d

"\n\n I don't see any specific issues about DACL discussed among customers."

In [130]:
query = "Give me specific issues discussed about compatibility"

In [126]:
context = get_context(db, query)

In [132]:
from pprint import pprint
pprint(context)

('work for you. (never tested on Cat 9200) - Cat 9300 ok\n'
 '\xa0\n'
 'Table 10.\xa0\xa0\xa0\xa0\xa0Network essentials and advantage package '
 'features\n'
 '\xa0\n'
 'https://www.cisco.com/c/en/us/products/collateral/switches/catalyst-9200-series-switches/nb-06-cat9200-ser-data-sheet-cte-en.html#Licensing\n'
 '\xa0 Does anyone know where to download :\n'
 'https://www.cisco.com/c/en/us/td/docs/security/ise/2-7/compatibility_doc/b_ise_sdt_27.html\n'
 'what kind of integration we are considering here ? |||| Hi Balaji,Thanks for '
 'the reply.We are considering\xa0Meraki MR series Access Points and MS series '
 'switches for\xa0Guest and dot1.x Authentications. Hello,\n'
 'Following the reset of the /opt partition which was full (100%) on MNT, we '
 'no longer have Splunk logs on 2 new PSN 3695\n'
 'while the other 4 PSN(3595) continue to work correctly.\n'
 'There are collection log Errors:\n'
 'The ISE MNT collector process is unable to persist the audit logs generated '
 'from the P

In [134]:
context = '''Prompt after formatting:
System: From the history and context provided below, Answer the question as truthfully as possible. 
    and if the answer is not contained within the context or history below, say 'I don't know' or ask me to refine my query. 

Current conversation:
[HumanMessage(content='Context:\n ```:\nhttps://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/guide-c07-656177.html\n\xa0 I have a customer who owns ISE admin but not FTD admin so they want to do dACL for RA VPN so they can skip the Access Control Policy on FTD. (So the less they have to ask to the FTD team base license?Thank you very much for your help and discussion!\xa0thanksJeffery\xa0\xa0\xa0\xa0 |||| Do you have RADIUS Accounting enabled on the ASA?\xa0 I\'m not sure of way to forcibly delete these (other than deleting the endpoint from context visibility maybe?).\xa0 What patch of 2.6 are you below\nhttps://bst.cisco.com/bugsearch/bug/CSCvw34491 Hi all,I have question please.Can I do accounting for device administration and without TACACS device admin license? |||| I mean Commands accounting. not endpoints accounting, my scope is to use ISE only for device administration Hello,I need actually working on a project integrating Dell (N Series) NAD\'s into ISE and would appreciate a clean copy of the switch config if you can provide it. I have the Cisco config but I would like to compare it to Dell\'sThanks! |||| Hello Leonardo,I\'m actually working on a project integrating Dell (N Series) NAD\'s into ISE and would appreciate a clean copy of the switch config if you can provide it. I have the Cisco config but I would like to compare it to Dell\'sThanks! |||| Hi Guilhermo,\xa0Follow the last firmware version that I\'ve implemented, using this version I also validated TACACS+ and Downloadable ACL via Dot1x, and works really fine.\xa0The script bellow does not have TACACS+ and dACL but you can find how to configure easily on manuals.\xa0!System Description "Dell EMC Networking N2048, 6.6.0.13, Linux 3.6.5-e3cd5a07, Not Available"!System Software Version 6.6.0.13!authentication enableauthentication dynamic-vlan enabledot1x system-auth-controlaaa authentication dot1x default radiusaaa (PassiveID)\nhttps://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/qa-c67-744190.pdf\xa0 |||| Gracias |||| Thank you.I\'ve been seeing Cisco\'s stuff, that\'s right. Thank you very much. Hi everyone!Can anyone consult me if there are any problems with ISE licensing in case I``` \n\n Query:\n```Give me all the issues about DACL discussed among customers. Provide them in bullet points.```', additional_kwargs={}, example=False), AIMessage(content="\n\n I don't see any specific issues about DACL discussed among customers.", additional_kwargs={}, example=False), HumanMessage(content='Context:\n ```FAQ : (is that helpful ?) - what features you looking, what kind of setup you have ?\n\xa0\n\xa0\nEssentials, Advantage, and Premier licenses in ISE 3.0\nhttps://www.cisco.com/c/en/us/products/collateral/security/identity-services-engine/qa-c67-744190.html Hi,\n\xa0\nI\'m preparing a solution for Guest WiFi with :\n\xa0\nhttps://community.cisco.com/kxiwq67737/attachments/kxiwq67737/4561-docs-security/6061/1/Integration_between_ISE_and_HPE-ArubaOS.pdf\nhttp://www.labminutes.com/sec0209_ise_20_3rd_party_nad_aruba_mab_dot1x_2\n\xa0\n\xa0 |||| there are many many timeout\xa0can I see\xa0show auth session for failed user interface\xa0 Hello,\n\xa0\nI\'m working on a project in order to deploy ISEs to gain more security. I would like to know what ise the stable OS version. Is it recommended to install version 3.1 or version 3.0? These ISEs will be deploy in production environnement.\n\xa0\nSincerly yours. |||| ISE 3.1 is the recommended version. So you should install that.Its from August 21, so got plenty of time in production many places, so its well tested.After installation of 3.1, remeber to install patches as well, as of this time patch3 |||| You need to define first what kind of deployment you are going to have: small, medium or large size deployment. We have a large size deployment with over 250K users, 10+ WLC\'s and we are about the deployment of Cisco ISE.Actually we are running a cluster of 2 members, each hosting PAN-MnT-PSN.We want to extend the ISE functions to our "industrial network" separated by a DMZ.To limit the trafic between the 2 zones, I would like to add one (or 2 for redundancy) server running only``` \n\n Query:\n```How about anything about deployment of Cisco ISE 3.0```', additional_kwargs={}, example=False), AIMessage(content='\n AI: \n\n The recommended version of Cisco ISE is 3.1. It has been in production use for several months and is well tested. After installation, it is important to install any available patches.', additional_kwargs={}, example=False), HumanMessage(content='Context:\n ```be something with the switche profile, but im still investigating does anyone have any ideas or solution for this.\n\xa0\n\nSaludos,\nGerardo Andree Mejía García |||| Hi Gerardo ,\xa0\nTo me it would appear that there\'s an issue of compatibility , please verify if you NAD is validated within the section "Validated Third Party Access Switches" in the following compatibility for ISE\xa0 https://www.cisco.com/c/en/us/td/docs/security/ise/3-0/compatibility_doc/b_ise_sdt_30.html#Cisco_Reference.dita_3456b3da-c198-439e-bb7c-d1ca839e1751\xa0as within those NAD,\xa0 it as been confirmed that the features of ISE work fine.\xa0\xa0 Hi Community,We are evaluating integration of meraki to our Cisco ISE deplyment with version 2.7 p7, but wanted to find out\xa0 if there are any limitations of integrating ISE with Meraki / any meraki version doesn\'t support or any feature limitations if any with cisco ISE.Any help or knowledge on this would be of great help.Thanks in advance. |||| chcek the matrix for support models previously but want to refresh this.We are having challenges on our network for authenticating MAC books using the JAMF and Cisco ISE integration.We see Cisco ISE provides various attributes like "MDM Manufacturer",\xa0MDM SerialNumber", "MDM DeviceRegisterStatus" etc. but none of these are working.\xa0We are trying to figure out what best will work in this case.\xa0\xa0With reference to the same checking for guidance from the community.\xa0\xa0Thanks!!N. |||| There is not enough information given to provide any meaningful assistance. Please see How to Ask the Community for Help .\nTroubleshooting MDM issues typically requires enabling the necessary debug logs, duplicating the issue, and reviewing the ise-psc.log file as per the Troubleshoot and Enable Debugs on ISE guide. If you\'re not familiar with troubleshooting ISE and need to resolve the issue quickly, you might be best opening a TAC case to investigate. Hi guys,\xa0I have fresh Cisco ISE 3.1 installation with patch1 and hotfix``` \n\n Query:\n```Give me specific issues discussed about deployment of Cisco ISE 3.0```', additional_kwargs={}, example=False), AIMessage(content="\n AI: \n I don't see any specific issues about deployment of Cisco ISE 3.0 discussed.", additional_kwargs={}, example=False)]

Context:
 ```work for you. (never tested on Cat 9200) - Cat 9300 ok
 
Table 10.     Network essentials and advantage package features
 
https://www.cisco.com/c/en/us/products/collateral/switches/catalyst-9200-series-switches/nb-06-cat9200-ser-data-sheet-cte-en.html#Licensing
  Does anyone know where to download :
https://www.cisco.com/c/en/us/td/docs/security/ise/2-7/compatibility_doc/b_ise_sdt_27.html
what kind of integration we are considering here ? |||| Hi Balaji,Thanks for the reply.We are considering Meraki MR series Access Points and MS series switches for Guest and dot1.x Authentications. Hello,
Following the reset of the /opt partition which was full (100%) on MNT, we no longer have Splunk logs on 2 new PSN 3695
while the other 4 PSN(3595) continue to work correctly.
There are collection log Errors:
The ISE MNT collector process is unable to persist the audit logs generated from the Policy Service nodes.
The current version is: 2.7.0.356 P5
BR,
...

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-134-b215d0ab45b8>, line 23)

In [137]:
llm(prompt=f"Context:\n ```{context}``` \n\n Query:\n```{query}```")

'There are a few issues discussed about compatibility. One issue is that the ISE MNT collector process is unable to persist the audit logs generated from the Policy Service nodes. This can cause problems with Splunk logs. Another issue is that some NADs may not be validated within the section "Validated Third Party Access Switches" in the compatibility for ISE. This can cause problems with the features of ISE working properly.'

In [42]:
from langchain import LLMChain, PromptTemplate
from steamship import Steamship
from langchain.chains import ConversationChain

In [36]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory 
from steamship_langchain.memory import ChatMessageHistory


In [43]:
template = """ Assistant is designed to be able to assist with a wide range of tasks, 
from answering simple questions to providing in-depth explanations and discussions on topics within conversation and context. 
As a language model, Assistant is able to generate human-like text based on the input it receives, 
allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. 
It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights.

If Assistant cannot infer any answer or response from previous conversation history or human input, say 'I don't know the answer'.

{history}

Human: {human_input}

Assistant:"""

# client = Steamship()
# prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)



chat_memory=ConversationBufferWindowMemory(k=3,return_messages=True)

chatgpt_chain = ConversationChain(
    llm=llm
    prompt=prompt,
    verbose=True,
    memory=chat_memory
)

chatgpt_chain.predict(
    human_input="I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply wiht the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd."
)

SyntaxError: invalid syntax (<ipython-input-43-c8645918af07>, line 26)

In [200]:
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [201]:
qa = ConversationalRetrievalChain.from_llm(llm, db.as_retriever(), memory=memory)

In [213]:
query = "Give me all failures in association with RSA"

In [228]:
t = db.similarity_search(query, k=5 )

In [229]:
context = ' '.join([page.page_content for page in t])

In [230]:
context

'SecurID\xa024551RSA request is declined, because RSA agent initialization has failed - RSA SecurID\xa024503Cannot establish a session with the RSA SecurID Server - RSA SecurID\xa022059The advanced option that is configured for process failure is used\xa022062The \'Drop\' advanced option is configured in case of able to esablish connection to RSA server, thus no logs on FW along the path.\nI would point my troubleshooting towards reconfiguration of ISE-RSA integration, like regenerate sdconf.rec file, checking DNS on both systems (are they able to resolve each other hostnames), and similar.\nBased on initialization has failed - RSA SecurID".13013Received TACACS+ Authentication START Request\xa015049Evaluating Policy Group\xa015008Evaluating Service Selection Policy\xa015048Queried PIP - DEVICE.Device Type\xa015041Evaluating Identity Policy\xa022072Selected identity source sequence - base license?Thank you very much for your help and discussion!\xa0thanksJeffery\xa0\xa0\xa0\xa0 |||| Do 

In [164]:
result = qa({"question": query})

In [165]:
result['answer']

'The issue is that the domain administrator user has permission to join AD, but the AD team is concerned that this user may be using LSA to perform read/write operations in AD.'

In [166]:
query = "Did they mention they found the solution?"
result = qa({"question": query})

In [167]:
result['answer']

'Not yet.'

In [172]:
query = "What was the recommended solution and who recommended it?"
result = qa({"question": query})

In [173]:
result['answer']

'There is not enough information given to provide any meaningful assistance. Please see How to Ask the Community for Help .'

In [174]:
result

{'question': 'What was the recommended solution and who recommended it?',
 'chat_history': [HumanMessage(content='Give me specific issue discussed about Active Directory', additional_kwargs={}, example=False),
  AIMessage(content='The issue is that the domain administrator user has permission to join AD, but the AD team is concerned that this user may be using LSA to perform read/write operations in AD.', additional_kwargs={}, example=False),
  HumanMessage(content='Did they mention they found the solution?', additional_kwargs={}, example=False),
  AIMessage(content='Not yet.', additional_kwargs={}, example=False),
  HumanMessage(content='How did their conversation end?', additional_kwargs={}, example=False),
  AIMessage(content='There is no answer to this question because the conversattion is still in progress.', additional_kwargs={}, example=False),
  HumanMessage(content='What was the recommended solution?', additional_kwargs={}, example=False),
  AIMessage(content='I will recommend

In [156]:
db.persist()
db=None

In [59]:
retriever = db.as_retriever()

In [60]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [79]:
query_text="""Can you give me a specific issue a user is having related to  to the user saying ```I cannot seem to get the credential scan from ISE working with a domain user '
 'account. I have tried multiple things, but I keep getting an error "The '
 'domain user account is invalid```. """

In [80]:
from pprint import pprint

In [81]:
pprint(qa.run(query_text))

(' The user tried to configure credential scan from ISE and got the error "The '
 'domain user account is invalid". |||| I would assume that the user has '
 'configured credential scan incorrectly.\n'
 'The error message is indicating that the domain user account is invalid. '
 'This means that the user may have entered the wrong username or password. It '
 'is also possible that the account does not exist in the domain.\n'
 'I would recommend that the user verify the username and password and ensure '
 'that the account exists in the domain. If the issue persists, I would '
 'contact the domain administrator for assistance.')


In [40]:
d = pd.read_csv('dataset/cluster_summaries.csv')

In [41]:
d.columns

Index(['topic_id', 'tag', 'summary', 'class', 'analysis_window',
       'quarter_for_analysis'],
      dtype='object')

In [45]:
d[(d['class'] == 'Upgrade') & (d['tag'] == 'Others')].head()

,topic_id,tag,summary,class,analysis_window,quarter_for_analysis
25,0,Others,* The Cisco 800 Series routers are experiencin...,Upgrade,annual,20233
26,1,Others,- The customer is having problems with their C...,Upgrade,annual,20233
46,0,Others,1. The customer is having trouble with their C...,Upgrade,quarterly,20233
47,1,Others,* The customer is having problems with their C...,Upgrade,quarterly,20233
